In [9]:
# linear programming problem
import pulp
min_problem = pulp.LpProblem("min ATE", pulp.LpMinimize)
max_problem = pulp.LpProblem("max ATE", pulp.LpMaximize)

# our hidden variables are the the probability of being in one of the partitions
partition_names = ['/'.join([compliance, response]) for compliance, response in partition_types]
q = {partition: pulp.LpVariable(partition, lowBound=0) for partition in partition_names}

In [10]:
# since our hidden vars are probabilities the sum of them should all be under 1
min_problem += sum([v for k,v in q.items()]) == 1

In [11]:
# statements
p_treatment_untreated_bad = q['never_taker/never_better'] + q['defier/never_better'] \
                             + q['never_taker/helped'] + q['defier/helped']

p_treatment_untreated_good = q['never_taker/always_better'] + q['defier/always_better'] \
                             + q['never_taker/hurt'] + q['defier/hurt']

p_treatment_treated_bad = q['always_taker/never_better'] + q['complier/never_better'] \
                             + q['always_taker/hurt'] + q['complier/hurt']

p_treatment_treated_good = q['always_taker/always_better'] + q['complier/always_better'] \
                             + q['always_taker/helped'] + q['complier/helped']

p_control_untreated_bad = q['never_taker/never_better'] + q['complier/never_better'] \
                             + q['never_taker/helped'] + q['complier/helped']

p_control_untreated_good = q['never_taker/always_better'] + q['complier/never_better'] \
                             + q['never_taker/hurt'] + q['complier/hurt']

p_control_treated_bad = q['always_taker/never_better'] + q['defier/never_better'] \
                             + q['always_taker/hurt'] + q['defier/hurt']

p_control_treated_good = q['always_taker/always_better'] + q['defier/always_better'] \
                             + q['always_taker/helped'] + q['defier/helped']

In [12]:
# there's a natural mapping from probabilities we see to the hidden variables we have
# we'll spell these out one by one.
# there are probably smarter ways to express this as a vector operation,
# but this is easier to understand
min_problem += p_treatment_untreated_bad == p_states['treatment/untreated/bad']

min_problem += p_treatment_untreated_good == p_states['treatment/untreated/good']

min_problem += p_treatment_treated_bad == p_states['treatment/treated/bad']

min_problem += p_control_untreated_bad == p_states['control/untreated/bad']

min_problem += p_control_untreated_good == p_states['control/untreated/good']

min_problem += p_control_treated_bad == p_states['control/treated/bad']

#min_problem += p_treatment_treated_good == p_states['treatment/treated/good']

#min_problem += p_control_treated_good == p_states['control/treated/good']

In [13]:
min_problem += q['complier/helped'] + q['defier/helped'] + q['always_taker/helped'] + q['never_taker/helped'] \
              - q['complier/hurt'] - q['defier/hurt'] - q['always_taker/hurt'] - q['never_taker/hurt']

In [14]:
pulp.LpStatus[min_problem.solve()]

'Optimal'

In [15]:
q_min = {partition:pulp.value(partition_p) for partition, partition_p in q.items()}
q_min

{'always_taker/always_better': 0.174,
 'always_taker/helped': 0.0,
 'always_taker/hurt': 0.316,
 'always_taker/never_better': 0.0,
 'complier/always_better': 0.0,
 'complier/helped': 0.03,
 'complier/hurt': 0.0,
 'complier/never_better': 0.028,
 'defier/always_better': 0.02,
 'defier/helped': 0.0,
 'defier/hurt': 0.0,
 'defier/never_better': 0.0,
 'never_taker/always_better': 0.0,
 'never_taker/helped': 0.0,
 'never_taker/hurt': 0.154,
 'never_taker/never_better': 0.278}